# Finding trees in LiDAR data
This is a simple example of what the PyCrown package can do: from pre-calculated rasters (CHM, DSM and DTM) and a height-normalized 3D LiDAR point cloud, individual trees can be segmented.
Outputs are shapefiles of tree top locations, crown shapes as well as a .LAS-file containing classified trees.

Installation Instructions for PyCrown
See: https://github.com/manaakiwhenua/pycrown


## Start with importing the modules

In [ ]:
import sys
from datetime import datetime
from pycrown import PyCrown


import rasterio as rio


## Set input files
Specify the file locations for the CHM, DSM, DTM and the LiDAR point cloud.
The latter is only needed if the point cloud should be classified into individual trees.

In [39]:
#Tell Python where the DTM and DSM Files that you processed from the point cloud are

F_DTM = 'data/DEM.tif'
F_DSM = 'data/DSM.tif'

#Open these Files Using rasterio

DTM_src = rio.open(F_DTM)
DSM_src = rio.open(F_DSM)

#read the rasters into an array

DTM_arr = DTM_src.read()
DSM_arr = DSM_src.read()

# print out the shape of the array

In [40]:
print(DTM_arr.shape)
print(DSM_arr.shape)

#subtract the DTM from the DSM to get the Canopy Height Model

CHM_arr = DSM_arr -DTM_arr

(1, 569, 1050)
(1, 569, 1050)


In [41]:
# Check that the shape is the same

print(CHM_arr.shape)

(1, 569, 1050)


In [42]:
# write the CHM model back to a raster object and save it

prof = DTM_src.profile

F_CHM = 'data/CHM.tif'

with rio.open(F_CHM, 'w', **prof) as dst:
        dst.write( (CHM_arr))




F_LAS = 'data/POINTS.las'

## Initialize an instance of PyCrown

In [43]:
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

## Clip all input data to new bounding box.

In [44]:
PC.clip_data_to_bbox((1785665, 1786715, 5477854, 5478423))

## Smooth CHM
A 5x5m block median filter is used (set circular=True to enable a disc-shaped window).

In [6]:
PC.filter_chm(5, ws_in_pixels=True, circular=False)

## Tree Detection with local maximum filter

In [7]:
PC.tree_detection(PC.chm, ws=5, hmin=16.)

## Clip trees to bounding box 
(no trees on image edge)
original extent: 1802140, 1802418, 5467295, 5467490    

In [8]:
PC.clip_trees_to_bbox(bbox=(1785665, 1786715, 5477854, 5478423))

## Crown Delineation

In [9]:
PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=15.,
                     th_seed=0.7, th_crown=0.55, max_crown=10.)

Tree crowns delineation: 0.002s


## (Optional) Correct tree tops on steep terrain

In [10]:
PC.correct_tree_tops()

Number of trees: 499
Tree tops corrected: 47
Tree tops corrected: 9.418837675350701%
DSM correction: 22
COM correction: 25


(22, 25)

## Calculate tree height and elevation

In [11]:
PC.get_tree_height_elevation(loc='top')
PC.get_tree_height_elevation(loc='top_cor')

## Screen small trees

In [12]:
PC.screen_small_trees(hmin=20., loc='top')

## Convert raster crowns to polygons

In [13]:
PC.crowns_to_polys_raster()

This command below produces an error but we can ignore it for this tutorial

In [ ]:
PC.crowns_to_polys_smooth(store_las=True)

## Check that all geometries are valid

In [15]:
PC.quality_control()

## Print out number of trees

In [16]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 417


## Export results

In [17]:
PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

In [18]:
PC.export_tree_locations(loc='top')

In [19]:
PC.export_tree_locations(loc='top_cor')

In [20]:
PC.export_tree_crowns(crowntype='crown_poly_raster')

This cell produces an error too but we can ignore it

In [ ]:
PC.export_tree_crowns(crowntype='crown_poly_smooth')

Now go to your c:\path to folder\pycrown-master\results folder to find the output


In [46]:
F_DEM_LID_RES = 'data/DEM_LID_RES.tif'
F_DSM_LID_RES = 'data/DSM_LID_RES.tif'
F_DSM_RPAS_RES = 'data/DSM_RPAS_RES.tif'

DEM_LID_RES_src = rio.open(F_DEM_LID_RES)
DSM_LID_RES_src = rio.open(F_DSM_LID_RES)
DSM_RPAS_RES_src = rio.open(F_DSM_RPAS_RES)

DEM_LID_RES_arr = DEM_LID_RES_src.read()
DSM_LID_RES_arr = DSM_LID_RES_src.read()
DSM_RPAS_RES_arr = DSM_RPAS_RES_src.read()

print(DEM_LID_RES_src.profile)
print(DSM_LID_RES_src.profile)
print(DSM_RPAS_RES_src.profile)


print(DEM_LID_RES_arr.shape)
print(DSM_LID_RES_arr.shape)
print(DSM_RPAS_RES_arr.shape)

#Correct the RPAS DEM for vertical offset of 12.4 m
OFFSET = 12.4
DSM_RPAS_RES_arr = DSM_RPAS_RES_arr - OFFSET

# USE THE LIDAR DEM LAYER TO CREATE a RPAS CHM
CHM_RPAS_RES_arr = DSM_RPAS_RES_arr - DEM_LID_RES_arr

print(CHM_RPAS_RES_arr.shape)


# write the  RPAS CHM model back to a raster object and save it

prof = DEM_LID_RES_src.profile

F_CHM_LID_RES = 'data/CHM_RPAS_RES.tif'

with rio.open(F_CHM_LID_RES, 'w', **prof) as dst:
        dst.write( (CHM_RPAS_RES_arr))

        
        

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32767.0, 'width': 1050, 'height': 680, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["D_NZGD_2000",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",173],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",1600000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(1.0, 0.0, 1785665.3468,
       0.0, -1.0, 5478423.7127), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -32767.0, 'width': 1050, 'height': 680, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["D_NZGD_2000",DATUM["unnamed",SPHEROID["unnamed",6378137,298.257222101004]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHOR

Special thanks to Andrew McMillan for perparing the lab session 